New Table for Weather yearly aggregates


In [1]:
import sqlite3

DB_FILE = "new_weather_data.db"

# Connect to SQLite database
conn = sqlite3.connect(DB_FILE)
cursor = conn.cursor()

# Create a table for yearly statistics
cursor.execute("""
    CREATE TABLE IF NOT EXISTS weather_yearly_summary (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        station_id TEXT NOT NULL,
        year TEXT NOT NULL,
        avg_max_temp REAL,
        avg_min_temp REAL,
        total_precipitation REAL,
        UNIQUE (station_id, year)
    );
""")

conn.commit()
conn.close()
print("Table 'weather_yearly_summary' created.")

Table 'weather_yearly_summary' created.


Computations

In [3]:
conn = sqlite3.connect(DB_FILE)
cursor = conn.cursor()

# Calculate yearly statistics, ignoring missing values (-9999)
cursor.execute("""
    INSERT OR REPLACE INTO weather_yearly_summary (station_id, year, avg_max_temp, avg_min_temp, total_precipitation)
    SELECT 
        station_id, 
        SUBSTR(date, 1, 4) AS year,
        AVG(NULLIF(max_temp, -9999)) / 10.0 AS avg_max_temp_c,  -- Convert to degrees Celsius
        AVG(NULLIF(min_temp, -9999)) / 10.0 AS avg_min_temp_c,  -- Convert to degrees Celsius
        SUM(NULLIF(precipitation, -9999)) / 100.0 AS total_precip_cm  -- Convert to centimeters
    FROM new_weather_data
    GROUP BY station_id, year;
""")

conn.commit()
conn.close()
print("Yearly weather aggregates computed and stored successfully.")


Yearly weather aggregates computed and stored successfully.


Results

In [4]:
import pandas as pd

conn = sqlite3.connect(DB_FILE)

# Read and display summary data as a dataframe
df = pd.read_sql_query("SELECT * FROM weather_yearly_summary ORDER BY station_id, year;", conn)
conn.close()

df


,id,station_id,year,avg_max_temp,avg_min_temp,total_precipitation
0,1,USC00110072,1985,15.334795,4.326446,78.01
1,2,USC00110072,1986,12.696337,2.176190,50.53
2,3,USC00110072,1987,17.760274,6.329863,79.36
3,4,USC00110072,1988,17.347268,4.534973,54.10
4,5,USC00110072,1989,15.651507,3.983562,79.37
...,...,...,...,...,...,...
4815,4816,USC00339312,2010,16.167671,4.334795,103.40
4816,4817,USC00339312,2011,16.307671,5.046027,136.07
4817,4818,USC00339312,2012,17.798361,5.165027,91.08
4818,4819,USC00339312,2013,15.473699,4.018356,109.90
